In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone https://github.com/rowanz/hellaswag

Cloning into 'hellaswag'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 45 (delta 0), reused 2 (delta 0), pack-reused 41
Unpacking objects: 100% (45/45), 17.69 MiB | 8.74 MiB/s, done.


In [3]:
!pip install torch transformers accelerate optimum

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.0/410.0 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.4 MB/s eta 0:00:00


In [13]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from datasets import load_dataset
from tqdm import tqdm
from optimum.bettertransformer import BetterTransformer
# Load your model and tokenizer
model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")
# convert the model to BetterTransformer
model = BetterTransformer.transform(model)

correct = 0  # Number of correct predictions
total = 0  # Total number of predictions

# Open the HellaSwag jsonl file
with open('/kaggle/working/hellaswag/data/hellaswag_val.jsonl', 'r') as f:
    # Loop through each line in the file
    for line in tqdm(f):
        item = json.loads(line)  # Parse each line as a JSON object

        context = item['ctx']  # Use 'ctx' field as context
        endings = item['endings']  # Possible endings
        correct_end = item['label']  # Correct answer index is in 'label' field

        # Generate a score for each possible ending
        scores = []
        for ending in endings:
            input_text = context + ' ' + ending  # Combine context and ending
#             print(input_text)
            input_ids = tokenizer.encode(input_text, return_tensors='pt').to("cuda")  # Tokenize input
            with torch.no_grad():
                outputs = model(input_ids, labels=input_ids)  # Get model output
            loss = outputs.loss  # Get loss value
            scores.append(loss.item())  # Add loss value to scores list

        # Choose the most likely ending based on the scores
        predicted_end = scores.index(min(scores))
        # Update the number of correct predictions
        if predicted_end == correct_end:
            correct += 1
        total += 1  # Update the total number of predictions

        # Print the accuracy
        print("Accuracy({}/{}): {}".format(correct, total, correct / total), end='\r')

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.
4it [00:00, 15.72it/s]

8it [00:00, 15.79it/s]

12it [00:00, 15.80it/s]

16it [00:01, 15.85it/s]

20it [00:01, 15.95it/s]

...

138it [00:08, 15.95it/s]

142it [00:09, 15.27it/s]

146it [00:09, 15.42it/s]

148it [00:09, 15.73it/s]


KeyboardInterrupt: 

In [4]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from datasets import load_dataset

from tqdm.auto import tqdm  # Using tqdm.auto for compatibility

# Load your model and tokenizer
model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")

# No need for BetterTransformer in this context. Remove the line.

correct = 0  # Number of correct predictions
total = 0  # Total number of predictions

# Open the HellaSwag jsonl file
with open('/kaggle/working/hellaswag/data/hellaswag_val.jsonl', 'r') as f:
  # Loop through each line in the file
  for line in tqdm(f, desc="Evaluating"):
    item = json.loads(line)  # Parse each line as a JSON object

    context = item['ctx']  # Use 'ctx' field as context
    endings = item['endings']  # Possible endings
    correct_end = item['label']  # Correct answer index is in 'label' field

    # Generate a score for each possible ending
    scores = []
    for ending in endings:
        input_text = context + ' ' + ending  # Combine context and ending
        input_ids = tokenizer.encode(input_text, return_tensors='pt').to("cuda")  # Tokenize input

        with torch.no_grad():
            outputs = model(input_ids, labels=input_ids)  # Get model output
            loss = outputs.loss  # Get loss value
            scores.append(loss.item())  # Add loss value to scores list

    # Choose the most likely ending based on the scores (lowest loss)
    predicted_end = scores.index(min(scores))

    # Update the number of correct predictions
    correct += 1 if predicted_end == correct_end else 0
    total += 1  # Update the total number of predictions

# Print the final accuracy after the loop
print(f"Final Accuracy: {correct / total:.4f}")


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Evaluating: 0it [00:00, ?it/s]

Final Accuracy: 0.3441
